# Timeliness, throughput and finality in Cardano Praos

This notebook is about the nitty-gritty details of how the performance of Cardano (in its Praos instantiation) pans out. 

Its narrative is from a _timeliness_ point of view. In taking this view it is aligning itself with likely user experience,
 throughput (or more precisely offered load) is seen primarily as something
that has an effect on the likely delivered timeliness. Not to say that throughput is not important, 
 but that throughput without timeliness has much less intrinsic value. 

It starts with a simple scenario - a sunny, gentle breeze, scenario - to explain the model.

It introduces which outcomes are important, where the boundaries are, it creates a 
 decomposition of the end-to-end process into key outcomes.
The decomposition structure is important, it is about creating boundaries around (bounded) 'doubt and uncertainty'.
In distributed systems (as, to be frank, in most of life) there are so many things that can 'go wrong', the boundaries that are
 established here help form a 'universe of discourse'.
Limiting the scope of concerns helps in the understanding of the 'couplings' - how the system's
 emergent timeliness, and other performance / resource use, interact.
Uniquely, the $\Delta Q$ approach, by incorporating the notion of non-occurrence in its framework, permits the inclusion of a 
  quantification of the effect of the concerns placed _outside_ the boundary. 
It is a framework that allows you to capture the uncertainties of your chosen framing of your problem within itself! 

The narrative includes use of monitoring data to show that the model and operational reality 'agree' - veracity 
 of any model is key if you wish to use it as a basis of extrapolation. 

It looks at scenarios with more offered load, windy days with gusts of various ferocity. 
It relates how those operational conditions change the timeliness of key steps and, for which steps the timeliness _does not change_. 
It also considers rainy-day scenarios, including what happens if there was an un-ending continuous downpour.

In this document we will use _throughput_ to refer to the effective throughput of the system, _load_ to 
 refer to the fraction of _offered load_ to the maximum _transported load_. 

It is a general property of systems (more precisely _systems of flow_) that their 'entropy' (the level of uncertainty around their predictability)
is at its highest when the offered load is equal to the transported load (often characterised as $\rho \to 1$ in the literature). 
The approach here is not to get sucked into characterising the system when $1 - \epsilon < \rho < 1 + \epsilon$, 
 as picking a fight with entropy is a hiding to nothing.
Instead, in designing the Cardano distributed systems we strove to make it 'unconditionally stable'; the system remains within 
 its _predictable region of operation_ irrespective of the offered load - its effective througphput does
 not degrade as the load increases, i.e. as $\rho \to \infty$. 
This gives a two-sided approach to bounding the maximum entropy case.

Many real world systems do not have this property.
This unconditional stablity (even in the presence of whole raft of classes of adversrial action)  was a goal from day-one of the design, 
 it permeates many detailed design decisions in the code, this was done as it is, in general, 
nigh-on impossible to retrofit such stablity into an already written system.

There is a lot of hype around the notion of 'transactions throughput' in blockchain systems. 
For the purposes of this narrative, transactions represent a unit of work that is submitted to the system 
  and (assuming the transaction is valid) appears in a on-chain block.
To account for how the various aspects of transaction create operational load - size, script execution and memory steps etc - 
  there is a cost model that endeavours to keep the worst case block processing costs 'the same' irrespective of transaction mixture.

For our purposes here the important point is that the Cardano system is designed to perform a certain amount of 'work',
 how that relates to transactions and their rate is dependent on the particular mixture being offered to the system.

## Scenario 1
### Low offered load, SPOs operating 'well', no adverserial action

There are several potential operating cases. The ususal situation is that the load is 'low' (not in saturation) - the offered load is below (over some reasonable timescale) that which the system can transport; the SPOs are operating 'well' (their clocks are reasonably in sync, they have allocated enough resources so that processing and propagation times are 'normal') and that there is no lurking adversery waiting in the wings to pounce (this also implies that they are generating blocks, so chain density is 'normal' i.e. $\ge 95\%$ of maximum impling there is no SPOs with significant stake offline or being silent preparing to become adversarial).

As said, this is the normal condition; in this situation transactions are included $\gg 99\%$ of the time in the next block (or two); forks are 'rare' (i.e. only due to _slot battles_ - where two block producers randomly create blocks at the same slot and the occasional (i.e. $\le 5$ per day) _height battle_ - where, due generated block temporal proximity, the previous block has not been diffused quickly enough to be used as the block to build on so a fork of length 1 occurs). Even where forks occur they are no bigger than size 1 and they are entirely resolved in less than $\Delta$; this also implies that block diffusion is $\ll \Delta$ ($5s$). No (properly formed, non-double spend attempt) transactions fail to make it into a block; the failure rate is effectivily $0$.

The emperical evidence for this is presented below (in section entitled _"Usual Operating Conditions - Emperical Evidence"_)


To be able to discuss the effects of less ideal conditions, we first consider this case and understand how the overall timeless (and theoretical potential transaction loss) acrues.

We will do this by considering 'observation points' - where (at a location) an event of significance can be both 'observed' and 'timed' (with, again for the purposes of this discussion, an accurate clock) - we call these 'observables'. 

### Identifying the key contributors
We start with the following collection of obserables for the Cardano system:

$\fbox{1}$  - tx submitted to cardano (e.g. over client2node or node2node interface)

$\fbox{2}$  - tx present in the mempool of the (eventual) block producer 
(the transaction has been diffused)

$\fbox{3}$ - block (containing the tx of interest) produced

$\fbox{4}$ - block diffused to 'all' (i.e. $\gg 95\%$) nodes

$\fbox{5}$ - block (containing the tx of interest) now 'old' enough to
be considered finalised ($\mathbb{P}[\text{block reverted}] < \epsilon$)


The above observables are related in a 'causal' fashion; the next observable in sequence can only occur _after_ the previous observable, namely:

$$
\fbox{1} \xrightarrow{\Delta Q^{1|\rightsquigarrow|2}} 
\fbox{2} \xrightarrow{\Delta Q^{2|\rightsquigarrow|3}} 
\fbox{3} \xrightarrow{\Delta Q^{3|\rightsquigarrow|4}} 
\fbox{4} \xrightarrow{\Delta Q^{4|\rightsquigarrow|5}} 
\fbox{5}
$$

where $\Delta Q^{A \rightsquigarrow B}$ is the _quality attentuation_, the delay and the possiblity of non-occurance of the observation $B$ occuring given that the observable $A$ has occurred.

Although this formulation may seem a bit convoluted, by posing the scenarios in this way we are able to exploit a mathematical framework where we can reason about both outcomes and variations in scenarios in both a compositional and sound way. In particular, when considering design variations, it creates a sound framework for identifying constraints (and logical infeasibilites) during the consideration of such variations. The formulation has the added benefit that it identifies the key metrics that should be captured in-life (where that is feasible) and provides a framing for tracking such ongoing performance against predicted norms.

### Defining measurements

We are now in a position to consider some of the measures being discussed in unambiguous terms.

#### Time To Mempool
In some ways only an 'omnisicient observer' could capture $\Delta Q^{1|\rightsquigarrow|2}$. Omnisicience is required as, by design, the identity (and hence the location) of the node that will create the next block is not knowable in advance. 

Practically, where nodes collaborate (as some do) in sharing their mempool contents, and the sampling represented by such a group of collaborating nodes was statistically significant, their observations would provide an approximation of both a lower and upper bound, namely $\left\lfloor\Delta Q^{1|\rightsquigarrow|2}\right\rceil$.

#### Time To Block
This is $\Delta Q^{1|\rightsquigarrow|3}$. If the submitter is willing to wait until the block is distributed, this value can be known (to within the precision of a slot interval) from recording the observable $\fbox{1}$ and deducing $\fbox{3}$ from the slot number of the transaction's block. It is this approach that the Cardano Foundation uses; submitting multiple 'canary' transactions per hour to create a longitudinal dataset.

If this metric is needed in more real-time fashion, then the issue of omnisicence arrises over which node will be the next block producer. 
So, in practice, the inclusion of the transaction has to be observed, say by the submitter or a trusted third party, which will occur at a point of time between $\fbox{3}$ and $\fbox{4}$, that being value $\le \Delta Q^{3|\rightsquigarrow|4}$. 

The value $\Delta Q^{1|\rightsquigarrow|3}$ is a sequential composition of $\Delta Q^{1|\rightsquigarrow|2}$ and $\Delta Q^{2|\rightsquigarrow|3}$ (we are assuming, at least for this initial explination, that $\Delta Q^{|\stackrel{\rightsquigarrow}{2}|}$ is very small and strictly bounded). In the $\Delta Q$ framework, such composition is simply the convolution of their values.

#### Time To Block Finalisation
We are going to define this as $\Delta Q^{3|\rightsquigarrow|5}$. The factors that effect this value is going to the major theme below.

## TBD
- build $\Delta Q$ cdfs for
  - time to chain (see timeliness doc)
  - time to next block (see timeliness doc)
  - convolve (they are going to be dominated by time to next block)
- discuss block diffusion
  - time to diffuse (single slot leader, network working normally) - mind your outcomes paper
  - slot battles (Praos Performance Model)
    - note model is for uniform distribution amongst the SPO (slot battle worst case)
      - note that battles represent "lost work opportunities" - best case is single SPO; worst case (as num (SPO) $\leadsto$ infinity) 5%
    - current environment (1000 or so SPO, not uniform) suggests about 2.5%, or ~540 slot battles per epoch or around 27 per 6 hr ([Current statistics](https://pooltool.io/networkhealth))
    - note that less SPOs is better for this metric, negative scalling effect _for this property_ 
  - height battles
    - cases by diffusion time
      - < 1s => next block has to be in next slot
      - < 3s => potential next blooks in next 3 sec
        - there are finite no of sequences that can occur, discuss them all
  - note that current sub-second (or thereabouts) diffusion happens due to _header pipelining_ and low block production and validation times
    - explain why header pipelining, why pipelining is single shot (until reset), why VRF as validation basis works, how _header wavefronts_ interact
    - explain why non-pipelining is much longer (note that this is the model in mind your outcomes paper)
  - current validation (benchmarking) for 'value' transactions is in the 2,800$\pm$300 per cpu second for adoption
    - work through benchmarking report?
- system under saturation (PRO discussion as intro)
  - back pressure becomes dominant
  - back pressure flow, pulsed filling of 'holes' created by blocks flowing outward

## Usual Operating Conditions - Empirical Evidence
 - measuring tx timeliness (use of Canaries)
 - Throughput measures
   - transactions per block (value movement - cardano TPS measure)
     - fiscal transactions per block
   - transactions per block (maximal script costs)
 - delay measures
   - block delays
   - temporal delays
 - no evidence of any transaction failure (in nnnn events);
 - discussion of the consequences of a 'poisson' service schedule



## Finalisation - transaction and block

 - forks have minimal effect tx finalisation if the tx is present in all forks
   - empirical measure of finalisation timing would be the same
   - equating block finalisation to tx finalisation could be seen as overly pessimistic, key issue is whether the transaction of interest ceases to be in the winning fork
     - if this is a 'random' process (i.e. not the direct effect of adversarial action where the adversarial nodes are deliberately excluding certain transactions) it can be modelled (derive model here)

 - 

## Other useful sources

- [Timeliness Document](https://docs.google.com/document/d/1B42ep9mvP472-s6p_1qkVmf_b1-McLNPyXGjGHEVJ0w)
- [Praos Performance Model](https://docs.google.com/document/d/1DdpSLtFbdp3B6UjwFYZ2MOUoQ0YvDqiy3veKzOfYGaQ)
- [Mind your outcomes](https://www.mdpi.com/2073-431X/11/3/45)
- [Discord article on Systems of Flow](https://medium.com/ten-thousand-tiny-revolutions/everything-is-a-system-of-flow-86066ebd4880&ved=2ahUKEwi0o_v0w-qJAxWBRUEAHYQnGwoQFnoECA0QAQ&usg=AOvVaw06x8imiEQdOa7gu8AsGN3B) concept is very general in systems engineering and queueing theory

# Transaction Progression in Saturation

Cardano's design was to create a distributed system with truly distributed, and decentralised, _peers_.
Decentralisation is often only seen as the mechanism that decides on the relationship between ledger blocks, 
in Cardano the nodes cooperate to create a semi-synchronous world-wide algorithm at several layers. 
This approach was taken so as to give the best operational conditions for minimising the computational
and communication footprint (as proxies for the overall energy consumption) for in the system.

This peer (and the resulting environmental footprint) is taken very seriously in the design, every `cardano-node` 
is a fully capable instance, the optimisation to various operational roles: edge node, central relay, block producer,
bootstrap node, dApp portal etc. being a matter of configuration. 
The contribution each node makes being defined by its configuration, connectivity (location and capacity) and 
computational (processing, memory and storage performance). 
Every node is equivalent to every other node, functionaly, though performance varies.

This may sound overly idealistic, but it has distinct advantages as it helps the design address issues of resilience, 
disincentivising (economically driven) centralisation and intermediation (market capture)
through creating lots of options of how an operational system can be realised.

There is nothing stopping an organisation taking the Cardano code base and, through configuration,
creating a very low cost (single-board computer),
highly resilient (no reliance on third party organisations for operational enviroments - 
over and above access to communications) global deployment.
All at a few tens of Watts of electrical power per node.

Such a system could have a few million users, perform a few tens to hundreds of millions of transactions per year,
be self organising (adapting automatically to nodes being added and removed and network connectivity changes),
resilient (capable of surving a plethora of 'disasters' up to and including a major subsea geological fault movement, 
Carrington Event or Coronal Mass Ejection - when suitably physically distributed).
In today's prices (2024) the core of such a system would have a capital 
cost $\$10,000$ to $\$20,000$ for, say, a five stakepool system that could
act as the core of a high reliablity, low operational cost store of distributed information.

There are many real-world problems that need such a reliable, robust, high-veracity distributed information store.

Cardano is a flexible, scalable, low operational cost system - its role in underpinning Ada and DeFi is just one use case.

## Timely diffusion as core functionality

Many blockchains fundementally rely on leading edge computational environments. 

Effectively they create a form of centralisation requiring the use of high-end data centres and/or large capital expenditure.
Having done this, they then apply their design focus on that highly controlled, expensive (in operational cost) systems and, because
of such focus, treat the distribution of transactions and blocks very much as a secondary consideration. 
In this way they have already given up on decentralisation, they have drunk the koolaid of (forced) intermediation and played into 
market capture and competitive exclusion of new, less captial rich, entrants.

This approach leads to viewing each system as a _contemporaneous and temporally consistent copy_ of blockchain's state with the distribution function being one of trying minmise inconsistency. 
Their ontology is one of every node having an identical view of some 'global' state, one that fundementally at odds with reality and realisability.

This total-global-consistent viewpoint of 'all the information I need in one place' can be beguiling. The assumption of _all relevant_ information being inside a single computational context (i.e. the high-end computer) creates false assumptions surrounding such issues as: transaction dependency, differential treatment through pricing/age/etc., front running, notions of fairness, etc.. 

Cardano's design does not take that design position. It recognises that communication takes time, it does not assume contemporaneous global consistency, it accepts that communications (and computations) are unreliable, it accepts that both transient (and more permanent) failure, it accepts that the underlying infrastructure changes (which may affect optimality).
It assumes that every node (except itself) has the potential to be, or become, adverserial.
It does all of this without _any transitive trust_, except for the information that has settled on-chain.

It approaches the issue of diffusion by having peers that are 'cold', 'warm' or 'hot' - collectively 'known' peers.
  - hot peers are ones that our node has a active TCP connection and a subscription to following the tip of the chain (of the remote peer).
  - warm peers are ones that our node has an active TCP connection, the remote peer may have a subscription to following our tip of chain.
  - cold peers are ones that are selected from the set of all possible peers; this selection is from
    - the ledger, weighted by stake
    - from peer-to-peer, by querying connected peers; although this is third party information it is not passed on until it has been connected to (i.e. a basic level of validation).
      Note: our node only supplies answers to peer-to-peer queries from its known peers.

The connectivity layer has targets, numbers of different temperature types (and other graduations, like assuring that a non-trival percentage of cold peers are from the ledger to avoid forms of sybil attacks) - all configurable:
  - hot: 10 to 20 (edge peers might have smaller number than this, at least 2)
  - warm: outgoing 20 to 30
    - incoming: 0 to 500 (depending on limits in configuration and load)
  - cold: 100

There are targets (except for incomming connections) that are actively maintained by a 'governor', numbers are maintained by promotion (in the case of cold, from ledger peers and exchanged peer-to-peer nodes). Churning also occurs to track optimality (on a hourly basis).

The hot peer numbers are chosen so as to make the possiblity of an accidental partition de-minimus (noting that there are also timeouts that reset connections to peers if they are inactive for long periods) - random graph theory suggests any number greater than 7 would be sufficient. SPOs are encouraged to run relays with higher numbers to create a variance in the inter-node valencies, this being a desirable property (from Small World graph theory). 

Optimality is achieved by ranking on the basis of timing of application level outcomes - header and block diffusion, the worst performing peers over a churn interval being replaced. 
The use of warm peers means that failed hot peers can be replaced in a single round trip time. The use of cold peers (in reasonably large numbers) helps mitigate both potential DNS issues as well as providing a basis of peer-to-peer addressing sharing. 

The high valency makes for a well connected network between peers where end-to-end hop counts are small, five or six maximum (both from Small World graphs and by measurement/observation). 
In a non-saturated network this creates the environment for a low end-to-end propergation delay while the block and transaction forwarding logic minimises the uneccessary duplicate transmision of information. 
The data diffusion protocols were designed to fit all notifications of new information into a single packet with the remote peer deciding which peer to get new blocks/transactions.
The aim is to get high resilience with near-optimal network resource use (both on a per-node and overall system basis).

## Saturation behaviour

Saturation behaviour is when the mempool is 'full' over many consecutive block servicing events, note that the mempool is sized such that it can hold
two block's worth of transactions, plus one transaction. 
This size was chosen so that a node can, in principle, have sufficient data already present to forge a next block in the next consecutive time slot. 

There are several costs associated with the size of the mempool:
  - for block forgers the mempool has to be revalidated as an initial step in block forging (to ensure that any transactions with exipred time-to-live are discarded)
  - for block adoptors (and forgers) the mempool has to be revalidated to ensure that 

A 'block's worth' in this context is by it meeting the 'full' criterion by any of the three measures:
 - representation size (octets)
 - script execution steps (nominal computational limit)
 - script memory use (nominal memory use)

For illustration (at time of writing 2024-12) a block can consist of anything from four (script step heavy) to 280+ (simple value movement) transactions.

In saturation the movement of transactions towards the next block forging node is triggered by space being made by the processing of the previous block. 
The system is 'pulsed'.

On the adoption of a block, a node will free up space in its local mem